# CLASIFICACIÓN
Desarrollar un modelo que prediga el porcentaje de grasa corporal de un individio como una variable continua.

## Regresión Lineal Múltiple

In [44]:
# Importar librerías necesarias
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.linear_model import LinearRegression
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

### Carga y análisis de datos

In [45]:
# Cargamos el CSV en un DataFrame
df = pd.read_csv("ObesityDataSet_raw_and_data_sinthetic.csv")

# Analisis exploratorio de los datos (EDA)
# print(df.info())
# print(df.describe())
# profile = ProfileReport(df)
# profile.to_file("obesidad.html")

# Longitud de los valores únicos de las variables para localizar las categóricas
for col in df.columns:
    print(f"{col}: {df[col].unique()}")
    print(f"{col}: {len(df[col].unique())}")

categoricas = ['family_history_with_overweight',  'CAEC', 'CALC', 'MTRANS', 'NObeyesdad']
binarias = ['Gender', 'FAVC', 'SMOKE' , 'SCC']

Gender: ['Female' 'Male']
Gender: 2
Age: [21 23 27 22 29 24 26 41 30 52 20 19 31 39 17 25 55 38 18 15 61 44 34 36
 32 35 40 45 33 51 56 28 16 37 14 42 43 46 48 47]
Age: 40
Height: [1.62 1.52 1.8  1.78 1.5  1.64 1.72 1.85 1.65 1.77 1.7  1.93 1.53 1.71
 1.69 1.6  1.75 1.68 1.58 1.79 1.56 1.67 1.66 1.81 1.82 1.55 1.61 1.63
 1.76 1.57 1.88 1.87 1.89 1.74 1.83 1.92 1.73 1.84 1.91 1.48 1.59 1.54
 1.9  1.86 1.45 1.98 1.51 1.46 1.49 1.95 1.94]
Height: 51
Weight: [ 64.    56.    77.   ... 131.41 133.69 133.35]
Weight: 1335
family_history_with_overweight: ['yes' 'no']
family_history_with_overweight: 2
FAVC: ['no' 'yes']
FAVC: 2
FCVC: [2.   3.   1.   2.45 2.88 2.01 2.6  2.59 2.39 1.12 2.03 2.66 2.89 2.71
 2.75 1.49 2.21 2.06 2.31 2.82 2.05 2.77 2.74 2.57 2.52 2.97 1.08 1.27
 1.34 2.96 2.73 2.84 2.44 2.43 2.93 2.02 1.03 1.59 1.21 1.52 2.7  2.36
 2.14 2.56 2.34 1.81 2.72 1.13 1.76 2.98 2.2  2.53 2.24 2.65 2.9  2.48
 2.95 2.78 1.01 2.94 1.89 1.95 2.28 2.37 2.32 2.62 1.05 2.76 2.07 2.69
 2.79 2.67 1.

### Conclusión del EDA
-   No se han encontrado valores nulos
-   No se aprecian valores atípicos los cuales no sean representativos
-   El dataset cuenta con con algunas filas duplicadas (0.4%)
-   ``Gender`` está altamamente correlacionada con ``Height`` y ``NObeyesdad``
-   ``Height`` está altamamente correlacionada con ``Gender``
-   ``NObeyesdad`` está altamamente correlacionada con ``Gender``, ``Weigth`` y ``family_history_with_overweight``
-   ``Weigth`` está altamamente correlacionada con ``NObeyesdad`` y ``family_history_with_overweight``
-   ``family_history_with_overweight`` está altamamente correlacionada con ``NObeyesdad`` y  ``Weigth``

### Limpieza de datos

In [46]:
# Eliminación de duplicados
print("Registros antes de eliminar duplicados: \n", df.count())
df = df.drop_duplicates()
print("Registros después de eliminar duplicados: \n", df.count())

Registros antes de eliminar duplicados: 
 Gender                            2111
Age                               2111
Height                            2111
Weight                            2111
family_history_with_overweight    2111
FAVC                              2111
FCVC                              2111
NCP                               2111
CAEC                              2111
SMOKE                             2111
CH2O                              2111
SCC                               2111
FAF                               2111
TUE                               2111
CALC                              2111
MTRANS                            2111
NObeyesdad                        2111
dtype: int64
Registros después de eliminar duplicados: 
 Gender                            2087
Age                               2087
Height                            2087
Weight                            2087
family_history_with_overweight    2087
FAVC                              2087
FCVC

In [47]:
# Creación de la columna IMC
df["IMC"] = df["Weight"] / (df["Height"] ** 2)

### Preprocesamiento y división de los datos


In [48]:
# Encodeamiento de las variables categóricas múltiples
for col in categoricas:
    df_encodeado = pd.get_dummies(df[col], drop_first=True)
    df = pd.concat([df, df_encodeado], axis=1)
df = df.drop(columns=categoricas)
print(df.info())
# Mapeo de las variables categóricas binarias
df["Gender"] = df["Gender"].map({'Male' : 0, 'Female' : 1})
df["FAVC"] = df["FAVC"].map({'no' : 0, 'yes' : 1})
df["SCC"] = df["SCC"].map({'no' : 0, 'yes' : 1})
df["SMOKE"] = df["SMOKE"].map({'no' : 0, 'yes' : 1})
print(df)

# Definimos las variables predictoras y objetivo
X = df.drop(columns=["IMC"])
y = df["IMC"]

# Escalamiento de las variables numéricas
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
print(f"Tamaño de X_scaled: {X_scaled.shape}, Tamaño de y: {y.shape}") # Asegurar que X_scaled y y tienen el mismo número de filas

# División de datos entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=42)

<class 'pandas.core.frame.DataFrame'>
Index: 2087 entries, 0 to 2110
Data columns (total 30 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Gender                 2087 non-null   object 
 1   Age                    2087 non-null   int64  
 2   Height                 2087 non-null   float64
 3   Weight                 2087 non-null   float64
 4   FAVC                   2087 non-null   object 
 5   FCVC                   2087 non-null   float64
 6   NCP                    2087 non-null   float64
 7   SMOKE                  2087 non-null   object 
 8   CH2O                   2087 non-null   float64
 9   SCC                    2087 non-null   object 
 10  FAF                    2087 non-null   float64
 11  TUE                    2087 non-null   float64
 12  IMC                    2087 non-null   float64
 13  yes                    2087 non-null   bool   
 14  Frequently             2087 non-null   bool   
 15  Sometimes

### Primer entrenamiento del modelo

In [49]:
# Calcular la correlación de las variables con IMC
correlaciones = df.corr()["IMC"].sort_values(ascending=False)

# Mostrar la correlación de cada variable con la variable objetivo
print("Correlaciones con la variable objetivo:")
print(correlaciones)

"""
· Si una variable tiene baja correlación con IMC (<0.1) → Puede no ser relevante y podríamos eliminarla.
· Si dos variables tienen correlación alta entre sí (>0.8) → Puede haber colinealidad y deberíamos eliminar una de ellas.
"""

# Calcular el VIF para cada variable independiente
vif_data = pd.DataFrame()
vif_data["Variable"] = X.columns
vif_data["VIF"] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]

# Mostrar los resultados
print("VIF de cada variable:")
print(vif_data.sort_values(ascending=False, by='VIF'))

"""
· VIF < 5 → No hay problema de colinealidad.
· VIF entre 5 y 10 → Hay colinealidad moderada, revisar.
· VIF > 10 → Fuerte colinealidad, se recomienda eliminar la variable.

"""

# Crear y entrenar el modelo de regresión lineal
modelo = LinearRegression()
modelo.fit(X_train, y_train)

# Obtener los coeficientes
coeficientes = pd.Series(modelo.coef_, index=X.columns)

# Mostrar el intercepto y los coeficientes
print("Intercepto:", modelo.intercept_)
print("Coeficientes:")
print(coeficientes.sort_values(ascending=False))

# Hacer predicciones en el conjunto de prueba
y_pred = modelo.predict(X_test)

# Calcular métricas de evaluación
mae = mean_absolute_error(y_test, y_pred)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

# Mostrar resultados
print(f"MAE (Error Absoluto Medio): {mae:.4f}")
print(f"MSE (Error Cuadrático Medio): {mse:.4f}")
print(f"RMSE (Raíz del Error Cuadrático Medio): {rmse:.4f}")
print(f"R² (Coeficiente de Determinación): {r2:.4f}")

Correlaciones con la variable objetivo:
IMC                      1.000000
Weight                   0.934242
Obesity_Type_III         0.668471
yes                      0.483593
Sometimes                0.419823
Obesity_Type_II          0.352815
FCVC                     0.265395
FAVC                     0.247461
Age                      0.241685
Sometimes                0.240533
CH2O                     0.143935
Obesity_Type_I           0.140068
Height                   0.122835
Public_Transportation    0.091849
Gender                   0.054779
NCP                      0.027889
SMOKE                   -0.002145
Bike                    -0.033241
Motorbike               -0.036328
Frequently              -0.064876
Overweight_Level_II     -0.077876
no                      -0.080939
TUE                     -0.104903
Walking                 -0.124143
FAF                     -0.183274
Overweight_Level_I      -0.185723
SCC                     -0.186924
no                      -0.221002
Normal_W

### Análisis del resultado
- 1. Verificación del tamaño de los datos
        
        ```
        
        Tamaño de X_scaled: (2087, 29), Tamaño de y: (2087,)
        ```
        
        ✅ **Todo bien**, `X_scaled` y `y` tienen la misma cantidad de filas (**2087 muestras**). Esto confirma que los datos están correctamente alineados.

- 2. Análisis de la Correlación con la Variable Objetivo (`IMC`)
    ```
    IMC                      1.000000
    Weight                   0.934242
    Obesity_Type_III         0.668471
    yes                      0.483593
    Sometimes                0.419823
    Obesity_Type_II          0.352815
    FCVC                     0.265395
    FAVC                     0.247461
    Age                      0.241685
    Sometimes                0.240533
    CH2O                     0.143935
    Obesity_Type_I           0.140068
    Height                   0.122835
    Public_Transportation    0.091849
    Gender                   0.054779
    NCP                      0.027889
    SMOKE                   -0.002145
    Bike                    -0.033241
    Motorbike               -0.036328
    Frequently              -0.064876
    Overweight_Level_II     -0.077876
    no                      -0.080939
    TUE                     -0.104903
    Walking                 -0.124143
    FAF                     -0.183274
    Overweight_Level_I      -0.185723
    SCC                     -0.186924
    no                      -0.221002
    Normal_Weight           -0.382134
    Frequently              -0.393167
    ```
    -   Deberiamos eliminar todas las variables que poseen una correlación negativa
    -   Se aprecia que la variable la cual  mas impacto a la hora de predecir el ``IMC`` es la variable ``Weight`` 

    ```
- 3. Verificación de la Multicolinealidad con VIF
    ```
    17              Sometimes  483.531430
    18                     no  457.044740
    16             Frequently   70.761094
    26       Obesity_Type_III   31.371086
    3                  Weight   26.827129
    25        Obesity_Type_II   20.745945
    24         Obesity_Type_I   14.177261
    28    Overweight_Level_II    7.970074
    14              Sometimes    5.894609
    2                  Height    5.686578
    27     Overweight_Level_I    5.418694
    13             Frequently    5.214409
    23          Normal_Weight    3.033141
    0                  Gender    2.540617
    1                     Age    2.279901
    21  Public_Transportation    2.095876
    15                     no    1.793737
    12                    yes    1.580115
    5                    FCVC    1.443324
    10                    FAF    1.299957
    4                    FAVC    1.290070
    22                Walking    1.289830
    6                     NCP    1.218741
    8                    CH2O    1.201822
    11                    TUE    1.190008
    9                     SCC    1.154218
    20              Motorbike    1.060976
    7                   SMOKE    1.055203
    19                   Bike    1.044510
    ```
    - Escala del VIF:
        - VIF < 5 → No hay problema de colinealidad.
        - VIF entre 5 y 10 → Hay colinealidad moderada, revisar.
        - VIF > 10 → Fuerte colinealidad, se recomienda eliminar la variable.

    - Eliminar todas las variables con VIF superior a 10.
- 4. Interpretación de los Coeficiente
    ```
    Weight                   6.818959
    Obesity_Type_III         2.245346
    Obesity_Type_II          1.325543
    Obesity_Type_I           1.239506
    Overweight_Level_II      0.807147
    Overweight_Level_I       0.661161
    Normal_Weight            0.322975
    yes                      0.162536
    no                       0.061303
    FCVC                     0.051655
    NCP                      0.050850
    FAVC                     0.029796
    Frequently               0.011944
    Motorbike                0.005074
    FAF                      0.004274
    no                      -0.000943
    TUE                     -0.004686
    Bike                    -0.008777
    Age                     -0.010500
    Walking                 -0.011265
    SMOKE                   -0.027993
    CH2O                    -0.047209
    SCC                     -0.050208
    Sometimes               -0.051471
    Sometimes               -0.052385
    Frequently              -0.110946
    Public_Transportation   -0.118083
    Gender                  -0.253270
    Height                  -2.467217
    ```
    - Eliminar las variables con coeficientes negativos
- 5. Evaluación del Modelo
    
    ```
    MAE (Error Absoluto Medio): 0.4552
    MSE (Error Cuadrático Medio): 0.3741
    RMSE (Raíz del Error Cuadrático Medio): 0.6116
    R² (Coeficiente de Determinación): 0.9943
    ```
    
    - MAE = 0.4552 → En promedio, el modelo se equivoca en 0.4552 unidades respecto al índice de masa corporal.
    - RMSE = 0.6116 Penaliza más los errores grandes
    - R² = 0.9943 -> El modelo explica el 99.4% de la variabilidad en el índice de masa corporal.
- 6. 🎯 ¿Es un buen modelo?
    - Un R² de 0.9943 indica que el modelo es muy preciso, ya que explica casi al 100% la variabilidad de la variable objetivo.
    - Podemos mejorar eliminando variables irrelevantes.

# Conclusión final: selección de modelo
- Métricas de la regresión lineal múltiple:
    ```
    MAE (Error Absoluto Medio): 0.4552
    MSE (Error Cuadrático Medio): 0.3741
    RMSE (Raíz del Error Cuadrático Medio): 0.6116
    R² (Coeficiente de Determinación): 0.9943
    ```

Debido a la falta de tiempo solo se ha podido realizar el entrenamiento de un modelo de regresión lineal múltiple. <br>
Aunque no se han eliminado las variables deseadas en el análisis posterior, hemos obtenido unos resultados bastante bueno ya que el modelo es capaz de explicar casi el 100% de la variabilidad del IMC. <br>
Por estas razones se ha elejido como mejor modelo. <br>
**Mejor modelo**: Regresión lineal múltiple